# Creating a Chatbot with Upstage Solar API Endpoint, Langchain and Gradio

This lab goes through the creation of a chatbot with the following details :
- Uses the Solar LLM API endpoint from Upstage.
- Uses Langchain for simplifying the API calls to Upstage Solar API and to also maintain context (chat history)
- Uses Gradio for the simple chatbot UI

![chatbot](../../../images/20240301-gradiochatbot.jpg)

### 1. Install the python modules

In [1]:
!pip install --upgrade -q langchain langchain-community langchainhub langchain-openai llama-index


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install --upgrade -q gradio


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### 2. Get the Solar API Endpoint key and setup the base url

Note that you will need to signup for the Solar API endpoint. It is in a free beta until the end of March 2024. From there, you can get your API key.

In [2]:
from getpass import getpass

BASE_URL = 'https://api.upstage.ai/v1/solar'
APIKEY = getpass()

### 3. Let's test out the Solar API with a simple chat API call

In [4]:
# Import the `ChatOpenAI` class from the `langchain_openai` package. 
# This class is used to interact with OpenAI's chat models.
from langchain_openai import ChatOpenAI

# Import `HumanMessage` and `SystemMessage` classes from the `langchain.schema` module.
# These classes are used to construct messages that simulate a chat interaction between a human and a system.
from langchain.schema import HumanMessage, SystemMessage

chat_model = ChatOpenAI(base_url=BASE_URL, model_name="solar-1-mini-chat", api_key=APIKEY)

# Create a list named `messages` containing instances of `SystemMessage` and `HumanMessage`.
messages = [
    SystemMessage(
        content="You are a helpful assistant."
    ),
    HumanMessage(
        content="Hi, how are you?"
    )
]

# Invoke the chat model using the `invoke` method of the `chat_model` object, passing the `messages` list as input.
response = chat_model.invoke(messages)

# Print the response received from the chat model to the console.
print(response)


content="Hello! I'm just a computer program, so I don't have feelings or emotions, but I'm here to help you with any questions or problems you may have. How can I assist you today?"


### 4. And here is the code for the full chatbot

- I think this code is pretty self explanatory.
- Note that it won't work necessarily in a cloud hosted notebook (like on Azure or maybe Google Colab). I don't think so anyway. The gradio chatbot assumes localhost. Personally, I did this with vscode and it worked quite well. Gardio runs very well on vscode.

In [3]:
import gradio as gr
import random
import time
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# chat_model sets up the atual chatbot API from upstage Solar
chat_model = ChatOpenAI(base_url=BASE_URL, model_name="solar-1-mini-chat", api_key=APIKEY)

# chat_history is a Langchain feature that helps manage chat history context
chat_history = ChatMessageHistory()

# we are using another langchain feature that creates a reusable prompt
# The first part of this defines the context for the chatbot and then 
# MessagesPlaceholder allows us to inject the chat history with the 
# users most recent prompt appended on. 
# This method continues to take messages and responses and accumulate
# context over the entire conversation.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# This is a big part of langchain, chaining together the prompt and the model (API)
chain = prompt | chat_model

# This doResponse function takes a user prompt and then
# injects the prompt (with some additional context) onto the chat history
# then sends the entire message history + prompt to the Solar API
# This function gets called by the Gradio Chatbot interface (see below)
def doResponse(user_prompt):
    global chat_model, chat_history

    chat_history.add_user_message(("(act as human friend to user. "
                                   "Don't act like an ai. "
                                   "Don't act like an assistant. " 
                                   "Do not break character) user prompt: ") + user_prompt)

    response = chain.invoke({"messages": chat_history.messages})

    chat_history.add_ai_message(response)
    return response.content


# The rest of this code uses Gradio to create a chatbot UI
# The Gradio interface calls the doResponse function with the 
# user's prompt and updates the chat window with the history
# that it is tracking itself.
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=300)
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, ch):
        time.sleep(2)
        bot_message = doResponse(message)
        ch.append((message, bot_message))
        return "", ch

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
